In [1]:
import pandas as pd
import numpy as np

# Limpeza de Dados

Dados do *mundo real* raramente são limpos: muitas vezes, para chegar na informação real precisamos penar e sofrer! A limpeza de dados (**data scrubbing** em inglês) é uma habilidade fundamental para qualquer pessoa trabalhando com dados (seja um cientista de dados, um analista de BI ou um engenheiro de dados!). Hoje vamos aprender algumas das técnicas que podemos utilizar para problemas gerais que encontramos em dados.

Vamos começar carregando nossos dados: utilizaremos hoje uma tabela semelhante à `tb_veic`: mas essa nova tabela contém mais informações! Infelizmente, muitas dessas informações estão com problemas, que tentaremos corrigir ao longo da aula hoje.

In [27]:
tb_veic_messy = pd.read_csv("data/vehicles_messy.csv", dtype={"year": "object"})


/var/folders/c2/4vv1dtt97gl9bn1lvw8rf0k40000gn/T/ipykernel_65685/1565233185.py:1: DtypeWarning: Columns (71,72,73,74,75,77,80,84) have mixed types. Specify dtype option on import or set low_memory=False.
  tb_veic_messy = pd.read_csv("data/vehicles_messy.csv", dtype={"year": "object"})


In [28]:
tb_veic_messy.head()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985a,1
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1


In [29]:
tb_veic_messy.shape

(37843, 86)

Como nosso DataFrame é muito longo (muitas colunas) não conseguimos vê-lo completamente através do método `.head()`. Para visualizar todas as colunas precisamos utilizar uma opção da biblioteca Pandas:

In [30]:
pd.set_option('display.max_columns', 500)
tb_veic_messy.head()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985a,1
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1


## Data types

Uma das primeiras etapas na limpeza de dados é garantir que as colunas tenham os tipos desejados: colunas numéricas devem ser ints e floats, colunas de data, datetime, strings serem objects... Primeiro vamos ver algumas formas de determinar os tipos de nossas colunas.

### Checando tipos

In [31]:
tb_veic_messy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37843 entries, 0 to 37842
Data columns (total 86 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       37843 non-null  int64  
 1   barrels08        37843 non-null  float64
 2   barrelsA08       37843 non-null  float64
 3   charge120        37843 non-null  float64
 4   charge240        37843 non-null  float64
 5   city08           37843 non-null  int64  
 6   city08U          37843 non-null  float64
 7   cityA08          37843 non-null  int64  
 8   cityA08U         37843 non-null  float64
 9   cityCD           37843 non-null  float64
 10  cityE            37843 non-null  float64
 11  cityUF           37843 non-null  float64
 12  co2              37843 non-null  int64  
 13  co2A             37843 non-null  int64  
 14  co2TailpipeAGpm  37843 non-null  float64
 15  co2TailpipeGpm   37843 non-null  float64
 16  comb08           37843 non-null  int64  
 17  comb08U     

In [32]:
tb_veic_messy.dtypes

Unnamed: 0           int64
barrels08          float64
barrelsA08         float64
charge120          float64
charge240          float64
                    ...   
phevCity             int64
phevHwy              int64
phevComb             int64
year_num_errado     object
mfrCodeDffr          int64
Length: 86, dtype: object

In [33]:
tb_veic_messy.dtypes.value_counts()

float64    32
int64      28
object     25
bool        1
dtype: int64

Além de acessar os tipos e conta-los, podemos utilizar o método `.select_dtypes()` para filtrar as colunas de um DataFrame de acordo com seu tipo:

In [34]:
tb_veic_messy.select_dtypes(include = 'object').head()

,drive,eng_dscr,fuelType,fuelType1,make,model,mpgData,trany,VClass,year,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,c240bDscr,createdOn,modifiedOn,startStop,year_num_errado
0,Rear-Wheel Drive,(FFS),Regular,Regular Gasoline,Alfa Romeo,Spider Veloce 2000,Y,Manual 5-spd,Two Seaters,1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,1985a
1,Rear-Wheel Drive,(GUZZLER),Regular,Regular Gasoline,Ferrari,Testarossa,N,Manual 5-spd,Two Seaters,1985,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,1985
2,Front-Wheel Drive,(FFS),Regular,Regular Gasoline,Dodge,Charger,Y,Manual 5-spd,Subcompact Cars,1985,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,1985
3,Rear-Wheel Drive,NaN,Regular,Regular Gasoline,Dodge,B150/B250 Wagon 2WD,N,Automatic 3-spd,Vans,1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,1985
4,4-Wheel or All-Wheel Drive,"(FFS,TRBO)",Premium,Premium Gasoline,Subaru,Legacy AWD Turbo,N,Manual 5-spd,Compact Cars,1993,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,1993


In [35]:
tb_veic_messy.select_dtypes(include = 'object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37843 entries, 0 to 37842
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   drive            36654 non-null  object
 1   eng_dscr         22440 non-null  object
 2   fuelType         37843 non-null  object
 3   fuelType1        37843 non-null  object
 4   make             37843 non-null  object
 5   model            37843 non-null  object
 6   mpgData          37843 non-null  object
 7   trany            37832 non-null  object
 8   VClass           37843 non-null  object
 9   year             37843 non-null  object
 10  guzzler          2281 non-null   object
 11  trans_dscr       15047 non-null  object
 12  tCharger         5186 non-null   object
 13  sCharger         666 non-null    object
 14  atvType          3072 non-null   object
 15  fuelType2        1408 non-null   object
 16  rangeA           1403 non-null   object
 17  evMotor          562 non-null  

### Convertendo tipos `.astype()`

Nas colunas acima podemos ver que a coluna `year` parece conter apenas números (o que faria sentido) mas está carregada com um `object` (o que não faz tanto sentido). Vamos investigar os dados dessa colunas para entendermos melhor o que ela contém:

In [36]:
tb_veic_messy['year'].head(20)

0     1985
1     1985
2     1985
3     1985
4     1993
5     1993
6     1993
7     1993
8     1993
9     1993
10    1993
11    1993
12    1993
13    1993
14    1985
15    1993
16    1993
17    1993
18    1993
19    1993
Name: year, dtype: object

In [37]:
tb_veic_messy['year'].describe()

count     37843
unique       34
top        1984
freq       1964
Name: year, dtype: object

In [38]:
tb_veic_messy['year'].unique()

array(['1985', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '1986', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '1984', '1987', '1988', '1989',
       '1990', '1991', '1992', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017'], dtype=object)

Aparentemente a colunas contém apenas números. Neste caso, podemos utilizar o método `.astype()` para converte-la em um tipo numérico:

In [39]:
tb_veic_messy.shape

(37843, 86)

In [40]:
tb_veic_messy['year_num'] = tb_veic_messy['year'].astype('int')

In [25]:
tb_veic_messy['year_num'].describe()

count    37843.000000
mean      2000.064398
std         10.390588
min       1984.000000
25%       1990.000000
50%       2001.000000
75%       2009.000000
max       2017.000000
Name: year_num, dtype: float64

### Convertendo Tipos com `.map()`

Além do método `.astype()`, podemos utilizar o método `.map()` para converter o tipo de uma coluna. O método `.map()` **mapea uma função sobre os elementos de uma coluna**, ou seja, ele retorna uma nova coluna, do tamanho da coluna original, preenchido com o resultado da função sobre os elementos da coluna original (um pouco parecido com os List Comprehensions...).

Vamos utilizar este método para aplicar a função `int` aos elementos da coluna `year`:

In [46]:
tb_veic_messy['year'].map(int)

0        1985
1        1985
2        1985
3        1985
4        1993
         ... 
37838    1993
37839    1993
37840    1993
37841    1993
37842    1993
Name: year, Length: 37843, dtype: int64

Porque usar o `.map()`? Em muitas situações, o problema em nossos dados não será apenas de definição de tipo e algumas linhas da nossa coluna numérica terão strings misturados... Neste caso o método `.astype()` não funcionará:

In [47]:
tb_veic_messy['year_num_errado'].unique()

array(['1985a', '1985', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '1986', '2002', 2002, 1986, 1985, 2003,
       2004, 2005, 1999, 2006, 2007, 2008, 1995, 1996, 2009, 2010, 1984,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1997, 1998, 2000,
       2001, 2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype=object)

In [48]:
tb_veic_messy['year_num_errado'].astype('int64')

ValueError: invalid literal for int() with base 10: '1985a'

Passar a função `int` através do `.map()` também não funcionará!

In [49]:
tb_veic_messy['year_num_errado'].map(int)

ValueError: invalid literal for int() with base 10: '1985a'

Mas o método `.map()` nos permite passar **QUALQUER FUNÇÃO** para sobre coluna! Vamos definir uma função que tenta converter um string em número mas que, ao invés de falhar com strings, retorna `np.nan`:

In [50]:
def te_int(value):
    try:
        return int(value)
    except ValueError as e:
        return np.nan

In [55]:
lista_num = [1, 2, 'a', '12', '15', 2.5]

In [56]:
[te_int(x) for x in lista_num]

[1, 2, nan, 12, 15, 2]

In [57]:
def te_int(value):
    try:
        return int(value)
    except ValueError as e:
        return np.nan

tb_veic_messy['year_map'] = tb_veic_messy['year_num_errado'].map(te_int)
tb_veic_messy[['year_num_errado', 'year_map']].head()

,year_num_errado,year_map
0,1985a,NaN
1,1985,1985.0
2,1985,1985.0
3,1985,1985.0
4,1993,1993.0


Quando verificamos os valores da coluna `year_num_errado` vimos que alguns anos estavam com um caráctere extra (a letra `a`). Com a função que definimos acima, nossa nova coluna de inteiros não terá nenhum valor nessas linhas. Como podemos alterar nossa função para que ela limpe, não remova, as linhas com `strings` misturados?

In [61]:
import re
value = '1985a'
pattern = r'[^0-9]'
int(re.sub(pattern, '', value))

1985

In [64]:
# EXERCICIO!
import re

def regex_year(value):
    return int(re.sub(r'[^0-9]', '', str(value)))

tb_veic_messy['year_regex'] = tb_veic_messy['year_num_errado'].map(regex_year)

In [65]:
tb_veic_messy['year_regex'].head()

0    1985
1    1985
2    1985
3    1985
4    1993
Name: year_regex, dtype: int64

## Valores ausentes (missing values)

Um dos maiores problemas que vamos enfrentar quando trabalhamos com dados reais são os ***missing values***. Os ***missing values*** surgem de 3 formas:

1. Pela aplicação de funções em nosso código 
    * **Exemplo**: `.map()` que construímos acima;
1. Através de falhas na coleta/armazenagem de dados
    * **Exemplo**: em uma pesquisa física, alguns campos preenchidos estavam ilegíveis.
1. Para representar características reais da observação
    * **Exemplo**: em uma tabela de e-commerce, com o faturamento mês a mês de cada item, a coluna `desconto` pode aparecer como NA para representar produtos que não estavam promocionados

Antes de tratarmos NAs precisamos **ENTENDER** porque eles existem, se é possível (e desejável) tentar recuperar essa informação, se ela pode ser reconstruída a partir de outras colunas, etc...

Começaremos vendo como a biblioteca Pandas representa NAs no Python: o objeto `np.nan`

### `np.nan` - o número que não é número!

Porque ter um tipo específico para NAs? Vamos utilizar o exemplo do `.map()` acima para entedermos essa escolha dos desenvolvedores: quando tentamos aplicar a função `int` sobre nossa coluna uma linha com o valor `1985a` impediu a aplicação da função! Uma linha em 37842! Imagine se NAs fossem `strings`: não consegueríamos aplicar nenhuma função numérica sobre colunas com mesmo só um NA!

In [69]:
type(np.nan)

float

In [74]:
np.nan + 1

nan

In [71]:
tb_veic_messy['year_map'].head()

0       NaN
1    1985.0
2    1985.0
3    1985.0
4    1993.0
Name: year_map, dtype: float64

In [75]:
tb_veic_messy.loc[0, 'year_map']

nan

In [76]:
tb_veic_messy.loc[0, 'year_map'] + 1

nan

In [77]:
tb_veic_messy['year_map'] + 1

0           NaN
1        1986.0
2        1986.0
3        1986.0
4        1994.0
          ...  
37838    1994.0
37839    1994.0
37840    1994.0
37841    1994.0
37842    1994.0
Name: year_map, Length: 37843, dtype: float64

Esse comportamento é um pouco diferente quando falamos de funções de agregação:

In [78]:
tb_veic_messy['year_map'].head()

0       NaN
1    1985.0
2    1985.0
3    1985.0
4    1993.0
Name: year_map, dtype: float64

In [79]:
tb_veic_messy['year_map'].mean()

2000.0647957296126

Um cuidado que devemos tomar é que quando utilizamos relação de equivalência com `np.nan`:

In [80]:
1 == 1

True

In [81]:
None == None

True

In [82]:
np.nan == np.nan

False

In [84]:
tb_veic_messy['year_map'].head()

0       NaN
1    1985.0
2    1985.0
3    1985.0
4    1993.0
Name: year_map, dtype: float64

In [86]:
tb_veic_messy['year_map'] == np.nan

0        True
1        True
2        True
3        True
4        True
         ... 
37838    True
37839    True
37840    True
37841    True
37842    True
Name: year_map, Length: 37843, dtype: bool

In [ ]:
tb_veic_messy[tb_veic_messy['year_map'] != np.nan].head()

### Os métodos `.isna()` e `.notna()`

Como vimos acima, para criar uma máscara representando valores `np.nan` não podemos utilizar uma relação de equivalência. Precisamos utilizar os métodos `.isna()` e `.notna()`!

In [87]:
tb_veic_messy['displ'].head()

0    2.0
1    4.9
2    2.2
3    5.2
4    2.2
Name: displ, dtype: float64

In [88]:
tb_veic_messy['displ'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 37843 entries, 0 to 37842
Series name: displ
Non-Null Count  Dtype  
--------------  -----  
37723 non-null  float64
dtypes: float64(1)
memory usage: 295.8 KB


In [89]:
tb_veic_messy['displ'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
37838    False
37839    False
37840    False
37841    False
37842    False
Name: displ, Length: 37843, dtype: bool

In [90]:
tb_veic_messy['displ'].notna()

0        True
1        True
2        True
3        True
4        True
         ... 
37838    True
37839    True
37840    True
37841    True
37842    True
Name: displ, Length: 37843, dtype: bool

In [91]:
mask_na_displ = tb_veic_messy['displ'].isna()
tb_displ_na = tb_veic_messy[mask_na_displ]

In [94]:
tb_displ_na.head()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro
7138,7138,0.240,0.0,0.0,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,85,0.0,0,0.0,40.0,0.0,0.0,NaN,NaN,NaN,0,NaN,-1,800,0,Electricity,Electricity,-1,-1,91,0.0,0,0.0,0.0,37.0,0.0,0,0,16423,0,0,Nissan,Altra EV,N,False,0,0,90,0.0,0.0,0.0,0.0,NaN,116.2069,0.0,129.6154,0.0,Midsize Station Wagons,2000,2750,NaN,NaN,NaN,NaN,EV,NaN,NaN,62 KW AC Induction,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Thu Jul 07 00:00:00 EDT 2016,N,0,0,0,2000,1,2000,2000.0,2000,1
7139,7139,0.282,0.0,0.0,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,72,0.0,0,0.0,47.0,0.0,0.0,NaN,NaN,2-Wheel Drive,0,NaN,-1,900,0,Electricity,Electricity,-1,-1,64,0.0,0,0.0,0.0,53.0,0.0,0,0,16424,0,0,Toyota,RAV4 EV,N,False,0,0,88,0.0,0.0,0.0,0.0,NaN,116.2069,0.0,91.0811,0.0,Sport Utility Vehicle - 2WD,2000,2250,NaN,NaN,NaN,NaN,EV,NaN,NaN,50 KW DC,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Thu Jul 07 00:00:00 EDT 2016,N,0,0,0,2000,1,2000,2000.0,2000,1
8143,8143,0.282,0.0,0.0,0.0,81,0.0,0,0.0,0.0,41.0,0.0,0,-1,0.0,0.0,72,0.0,0,0.0,47.0,0.0,0.0,NaN,NaN,2-Wheel Drive,0,NaN,-1,900,0,Electricity,Electricity,-1,-1,64,0.0,0,0.0,0.0,53.0,0.0,0,0,17328,0,0,Toyota,RAV4 EV,N,False,0,0,88,0.0,0.0,0.0,0.0,NaN,116.2069,0.0,91.0811,0.0,Sport Utility Vehicle - 2WD,2001,2250,NaN,NaN,NaN,NaN,EV,NaN,NaN,50 KW DC,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Thu Jul 07 00:00:00 EDT 2016,N,0,0,0,2001,1,2001,2001.0,2001,1
8144,8144,0.312,0.0,0.0,0.0,74,0.0,0,0.0,0.0,46.0,0.0,0,-1,0.0,0.0,65,0.0,0,0.0,52.0,0.0,0.0,NaN,NaN,NaN,0,NaN,-1,1000,0,Electricity,Electricity,-1,-1,58,0.0,0,0.0,0.0,59.0,0.0,0,0,17329,0,0,Ford,Th!nk,N,False,0,0,29,0.0,0.0,0.0,0.0,NaN,105.3125,0.0,82.1951,0.0,Two Seaters,2001,1750,NaN,NaN,NaN,NaN,EV,NaN,NaN,27 KW AC Induction,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Thu Jul 07 00:00:00 EDT 2016,N,0,0,0,2001,1,2001,2001.0,2001,1
8146,8146,0.522,0.0,0.0,0.0,45,0.0,0,0.0,0.0,75.0,0.0,0,-1,0.0,0.0,39,0.0,0,0.0,87.0,0.0,0.0,NaN,NaN,2-Wheel Drive,0,NaN,-1,1700,0,Electricity,Electricity,-1,-1,33,0.0,0,0.0,0.0,102.0,0.0,0,0,17330,0,0,Ford,Explorer USPS Electric,N,False,0,0,38,0.0,0.0,0.0,0.0,NaN,62.4074,0.0,46.8056,0.0,Sport Utility Vehicle - 2WD,2001,-1750,NaN,NaN,NaN,NaN,EV,NaN,NaN,67 KW AC Induction,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Thu Jul 07 00:00:00 EDT 2016,N,0,0,0,2001,1,2001,2001.0,2001,1


In [97]:
sum(tb_veic_messy['displ'].isna())

120

In [98]:
tb_veic_messy['displ'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
37838    False
37839    False
37840    False
37841    False
37842    False
Name: displ, Length: 37843, dtype: bool

### O método `.isna()` em DataFrames

Além de sua utilização em `Series`, o método `.isna()` pode ser utilizado em DataFrames: o resultado será um DataFrame de máscaras com a avaliação do método aplicada à cada coluna:

In [99]:
tb_veic_messy.isna()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,False,True,False,False,True,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,True,True,True,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,True,True,True,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37838,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

Agora podemos utilizar o método `.sum(axis = 1)` para contabilizar o número de NAs em cada linha!

In [100]:
tb_veic_messy.isna().sum(axis = 1)

0        13
1        11
2        11
3        13
4        11
         ..
37838    11
37839    12
37840    11
37841    12
37842    10
Length: 37843, dtype: int64

In [101]:
tb_veic_messy['num_val_na'] = tb_veic_messy.isna().sum(axis = 1)

In [106]:
tb_veic_messy.isna().sum(axis = 1)

0        13
1        11
2        11
3        13
4        11
         ..
37838    11
37839    12
37840    11
37841    12
37842    10
Length: 37843, dtype: int64

In [107]:
tb_veic_messy['num_val_na'].describe()

count    37843.000000
mean        11.312581
std          1.288592
min          4.000000
25%         11.000000
50%         12.000000
75%         12.000000
max         14.000000
Name: num_val_na, dtype: float64

Com essa nova coluna podemos construir filtros sobre a **completude** das linhas, ou seja, quantas colunas dela tem valores missing:

In [112]:
mask_num_na = tb_veic_messy['num_val_na'] < 5
mask_num_na

0        False
1        False
2        False
3        False
4        False
         ...  
37838    False
37839    False
37840    False
37841    False
37842    False
Name: num_val_na, Length: 37843, dtype: bool

In [113]:
tb_veic_messy[mask_num_na].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 29924 to 30453
Data columns (total 91 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2 non-null      int64  
 1   barrels08        2 non-null      float64
 2   barrelsA08       2 non-null      float64
 3   charge120        2 non-null      float64
 4   charge240        2 non-null      float64
 5   city08           2 non-null      int64  
 6   city08U          2 non-null      float64
 7   cityA08          2 non-null      int64  
 8   cityA08U         2 non-null      float64
 9   cityCD           2 non-null      float64
 10  cityE            2 non-null      float64
 11  cityUF           2 non-null      float64
 12  co2              2 non-null      int64  
 13  co2A             2 non-null      int64  
 14  co2TailpipeAGpm  2 non-null      float64
 15  co2TailpipeGpm   2 non-null      float64
 16  comb08           2 non-null      int64  
 17  comb08U     

### Calculando a % de Valores NA

Utilizando o que construímos até agora, podemos construir uma coluna com a informação da % de linhas que apresentam valores NAs.

In [120]:
tb_veic_messy.isna().sum(axis = 1)/tb_veic_messy.shape[1]

0        0.141304
1        0.119565
2        0.119565
3        0.141304
4        0.119565
           ...   
37838    0.119565
37839    0.130435
37840    0.119565
37841    0.130435
37842    0.108696
Length: 37843, dtype: float64

In [121]:
tb_veic_messy['per_val_na'] = tb_veic_messy.isna().sum(axis = 1)/tb_veic_messy.shape[1]

In [122]:
tb_veic_messy['per_val_na'].describe()

count    37843.000000
mean         0.122963
std          0.014006
min          0.043478
25%          0.119565
50%          0.130435
75%          0.130435
max          0.152174
Name: per_val_na, dtype: float64

## Removendo (***dropando***) colunas 
Como vimos antes, o método `.drop()` pode ser utilizado para remover linhas ou colunas. Quando queremos remover uma coluna de nosso tabela devemos usar um dois dois argumentos abaixo:

1. `axis=1` ou
1. `columns = ['nome_da_coluna1', 'nome_da_coluna2' ,...]`

In [135]:
tb_veic_messy.drop('year', axis = 1).head()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na,per_val_na
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,-1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985a,1,1985,NaN,1985,1,13,None
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,-8500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11,None
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,500,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11,None
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,-8500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,13,None
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,-4000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1,1993,1993.0,1993,1,11,None


In [133]:
tb_veic_messy_noyear = tb_veic_messy.drop(columns = ['year']).head()

### Removendo colunas condicionalmente

Uma técnica importante é a remoção de colunas através de condições. Por exemplo, nosso DataFrame pode ter colunas com um único valor. Vamos utilizar o método `.drop()` para remover essas colunas:

In [136]:
tb_veic_messy.columns

Index(['Unnamed: 0', 'barrels08', 'barrelsA08', 'charge120', 'charge240',
       'city08', 'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF',
       'co2', 'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'guzzler',
       'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr',
       'createdOn

In [139]:
len(tb_veic_messy['per_val_na'].unique())

1

In [140]:
colunas_unique = []
for coluna in tb_veic_messy.columns:
    n_valores = len(tb_veic_messy[coluna].unique())
    if n_valores == 1:
        colunas_unique.append(coluna)
print(colunas_unique)

['charge120', 'mfrCodeDffr', 'pedro', 'per_val_na']


Vamos praticar nosso raciocinio funcional transformando o loop acima em um list comprehension!

In [141]:
# EXERCICIO
[coluna for coluna in tb_veic_messy.columns if len(tb_veic_messy[coluna].unique()) == 1]

['charge120', 'mfrCodeDffr', 'pedro', 'per_val_na']

Agora, com nossa lista de colunas, podemos utilizar o método `.drop()` para tratar nosso DataFrame:

In [145]:
tb_veic_messy_nopedro = tb_veic_messy.drop(columns = colunas_unique)

## O método `.dropna()`

A maneira mais **direta**, *mas nem sempre correta*, de tratar NAs é removendo-os da nossa tabela. Como podemos saber quando remover NAs? Basta lembrar dos três motivos pelo qual eles aparecem:

1. Pela aplicação de funções em nosso código 
    * **Exemplo**: `.map()` que construímos acima;
1. Através de falhas na coleta/armazenagem de dados
    * **Exemplo**: em uma pesquisa física, alguns campos preenchidos estavam ilegíveis.
1. Para representar características reais da observação
    * **Exemplo**: em uma tabela de e-commerce, com o faturamento mês a mês de cada item, a coluna `desconto` pode aparecer como NA para representar produtos que não estavam promocionados

Na sua opinião, qual tipo de NA devemos manter e qual devemos excluir da base?

Uma vez determinado que vamos remover NAs temos duas formas de fazê-lo: excluindo colunas e excluindo linhas. Em geral excluímos colunas quando a quantidade de NAs é tão grande que seria impossível utilizar a informação da coluna para qualquer coisa. No caso das linhas, removemos linhas com NA **APENAS QUANDO AS COLUNAS DE NOSSA ANÁLISE FOREM IMPACTADAS!!**

Vamos ver como o método `.dropna()` pode nos ajudar aqui.

Primeiro em tudo:

In [147]:
tb_veic_messy.dropna()

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na,per_val_na


Agora nas linhas:

In [148]:
tb_veic_messy.dropna(axis = 0)

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na,per_val_na


Agora nas colunas:

In [149]:
tb_veic_messy.dropna(axis = 1)

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,engId,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,charge240b,createdOn,modifiedOn,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_regex,pedro,num_val_na
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,9011,-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985a,1,1985,1985,1,13
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,22020,-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985,1,11
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,2100,-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985,1,11
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,2850,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985,1,13
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,66031,-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37838,37838,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,66030,-1,1500,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-750,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993,1,11
37839,37839,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,66030,-1,1450,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,25.0000,0.0,39.0000,0.0,Compact Cars,1993,-500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993,1,12
37840,37840,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,66030,-1,1600,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,0.0,0.0,0.0,0,0,9997,0,14,Subaru,Legacy AWD,Y,False,0,90,0,0.0,0.0,0.0,0.0,23.0000,0.0,34.0000,0.0,Compact Cars,1993,-1250,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993,1,11
37841,37841,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,

Independente do eixo (`axis`) de aplicação, podemos especificar o `thresh` - um patamar de NAs abaixo do qual removemos uma linha ou coluna. Se escolhermos o eixo das linhas, e colocarmos o `thresh = 60`, manteremos apenas as linhas com pelo menos 60 colunas preenchidas. Já no caso das colunas, o `thresh` será o mínimo de linhas não-NA que uma coluna precisar para continuar no DataFrame.

In [153]:
tb_veic_messy.dropna(axis = 0, thresh = 10)

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na,per_val_na
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985a,1,1985,NaN,1985,1,13,None
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11,None
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11,None
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,13,None
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1,1993,1993.0,1993,1,11,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37838,37838,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1500,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-750,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,N

In [154]:
tb_veic_messy.dropna(axis = 1, thresh = 5000)

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,trans_dscr,tCharger,mfrCode,charge240b,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,NaN,NaN,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985a,1,1985,NaN,1985,1,13
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,NaN,NaN,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,SIL,NaN,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,11
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,NaN,NaN,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1985,1,1985,1985.0,1985,1,13
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,NaN,T,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1,1993,1993.0,1993,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37838,37838,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1500,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-750,CLKUP,NaN,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0,1993,1,1993,1993.0,1993,1,11
37839,37839,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1450,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0000,0.0,

In [155]:
thresh_col = int(tb_veic_messy.shape[0] * 0.20)
thresh_col

7568

In [156]:
tb_veic_messy.dropna(axis = 1, thresh = thresh_col)

,Unnamed: 0,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,trans_dscr,charge240b,createdOn,modifiedOn,phevCity,phevHwy,phevComb,year_num_errado,mfrCodeDffr,year_num,year_map,year_regex,pedro,num_val_na
0,0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1250,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985a,1,1985,NaN,1985,1,13
1,1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3050,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-8500,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985.0,1985,1,11
2,2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1250,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,500,SIL,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985.0,1985,1,11
3,3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3050,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-8500,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1985,1,1985,1985.0,1985,1,13
4,4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2150,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-4000,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993.0,1993,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37838,37838,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1500,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-750,CLKUP,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,1993,1,1993,1993.0,1993,1,11
37839,37839,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1450,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0000,0.0,39.0000,0.0,Compact Cars,1993,-500,NaN,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0,19

## O método `.fillna()`

Em algumas condições (quais?) é melhor substituir os ***missing values*** por algum outro valor. Para tanto vamos utilizar o método `.fillna()`.

In [157]:
tb_veic_messy['trans_dscr'].isna().sum(axis = 0)

22796

In [158]:
tb_veic_messy['trans_dscr'].unique()

array([nan, 'SIL', '2MODE CLKUP', 'CLKUP', '2MODE', '3MODE CLKUP',
       '2MODE 2LKUP', '3MODE', '4MODE', 'Creeper', 'SIL Creeper',
       'EMS 2MODE', '2LKUP', '2MODE CLKUP FW', 'DC/FW', 'EMS',
       '2MODE VLKUP', 'EMS 2MODE CLKU', '3MODE 2LKUP', 'EMS 3MODE',
       'VMODE CLKUP', 'VLKUP', 'EMS 2MODE CLKUP', 'EMS CLKUP',
       'EMS CMODE CLKUP', 'SIL CLKUP', 'CMODE CLKUP', 'VMODE', 'CMODE',
       '3LKUP', 'EMS 3MODE CLKUP', 'SIL EMS', '6MODE', '4MODE CLKUP',
       'VMODE VLKUP', '6MODE CLKUP', 'SIL 3MODE', 'LONG RATIO',
       'CMODE VLKUP', 'SIL 3MODE CLKUP', '3MODE VLKUP', 'Overdrive',
       'Lockup', '3MODE CLKUP FW', 'SIL 2MODE CLKUP', 'EMS 5MODE',
       '2MODE DC/FW', 'fuel injection', '2MODE 3LKUP', 'Lockup A3',
       'SIL CMODE', 'Elec Overdrive', 'Mech Overdrive'], dtype=object)

In [159]:
tb_veic_messy['trans_dscr'].fillna('MANUAL')

0        MANUAL
1        MANUAL
2           SIL
3        MANUAL
4        MANUAL
          ...  
37838     CLKUP
37839    MANUAL
37840     CLKUP
37841    MANUAL
37842     CLKUP
Name: trans_dscr, Length: 37843, dtype: object

In [160]:
tb_veic_messy['trans_dscr_fna'] = tb_veic_messy['trans_dscr'].fillna('MANUAL')

In [161]:
tb_veic_messy['trans_dscr_fna'].value_counts()

MANUAL             22796
CLKUP               7809
SIL                 2189
2MODE CLKUP         1235
Creeper              525
EMS 2MODE CLKUP      520
3MODE CLKUP          517
2MODE                448
2MODE 2LKUP          383
EMS                  252
3MODE                166
CMODE                150
CMODE CLKUP          130
VMODE CLKUP          105
SIL Creeper           72
2LKUP                 63
DC/FW                 53
VLKUP                 52
EMS CLKUP             50
EMS 2MODE             46
6MODE CLKUP           41
4MODE                 35
VMODE VLKUP           26
2MODE 3LKUP           22
2MODE DC/FW           19
CMODE VLKUP           17
3LKUP                 15
EMS 3MODE             11
Overdrive             10
Lockup                 9
2MODE VLKUP            9
SIL EMS                7
6MODE                  6
SIL 3MODE              6
4MODE CLKUP            6
3MODE VLKUP            5
SIL 2MODE CLKUP        5
EMS 2MODE CLKU         5
Elec Overdrive         3
Mech Overdrive         3


In [162]:
tb_veic_messy\
    .select_dtypes(include = "number")\
    .isna().sum(axis = 0)\
    .sort_values(ascending=False)

cylinders      123
displ          120
year_map         1
Unnamed: 0       0
lv4              0
              ... 
fuelCostA08      0
ghgScore         0
ghgScoreA        0
highway08        0
num_val_na       0
Length: 65, dtype: int64

In [166]:
tb_veic_messy['cylinders'].median()

6.0

Podemos utilizar a média de uma coluna numérica como substituto para valores NAs. Essa técnica chama-se imputação de valores nulos, e, no módulo 3 veremos formas menos rudimentares de utiliza-la!

In [165]:
tb_veic_messy['cyl_impu'] = tb_veic_messy['cylinders'].fillna(tb_veic_messy['cylinders'].median())
tb_veic_messy['cyl_impu'].unique()

array([ 4., 12.,  8.,  6.,  5., 10.,  2.,  3., 16.])

In [167]:
tb_veic_messy[['cyl_impu', 'cylinders']].describe()

,cyl_impu,cylinders
count,37843.000000,37720.000000
mean,5.738842,5.737990
std,1.749010,1.751795
min,2.000000,2.000000
25%,4.000000,4.000000
50%,6.000000,6.000000
75%,6.000000,6.000000
max,16.000000,16.000000


## Duplicação de linhas

Em algumas situações, nossos DataFrames podem conter linas duplicadas. Para duplicações simples (onde todas as colunas são iguais) podemos utilizar o método `.duplicated()` para limpar as duplicatas.

In [168]:
frutas = pd.DataFrame({
    'fruta':['laranja', 'laranja', 'mamão', 'laranja'],
    'cidade':['Rio de Janeiro', 'Atibaia', 'Campinas', 'Rio de Janeiro']
})

In [169]:
frutas

,fruta,cidade
0,laranja,Rio de Janeiro
1,laranja,Atibaia
2,mamão,Campinas
3,laranja,Rio de Janeiro


In [170]:
frutas.duplicated()

0    False
1    False
2    False
3     True
dtype: bool

In [171]:
frutas.loc[~frutas.duplicated(),]

,fruta,cidade
0,laranja,Rio de Janeiro
1,laranja,Atibaia
2,mamão,Campinas


# 21h31